In [110]:
import pandas as pd
from sklearn.metrics import (
    average_precision_score,
    roc_auc_score,
)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [111]:
cdr1 = list(range(27, 38 + 1))
cdr2=list(range(56, 65 + 1))
cdr3=list(range(105, 117 + 1))
cdrs = cdr1 + cdr2 + cdr3
def get_ap_scores(predictions):
    cdr_ranges = {
    "cdr1": cdr1,
    "cdr2": cdr2,
    "cdr3": cdr3
    }
    ap_scores = {}

    for cdr_name, cdr_range in cdr_ranges.items():
        preds_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='light'")["prediction"].tolist()
        labs_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='light'")["labels"].tolist()
        ap_scores[cdr_name] = average_precision_score(labs_cdr, preds_cdr)

    # Display the AP scores for each CDR
    print("Average Precision Scores by CDR:")
    for cdr, score in ap_scores.items():
        print(f"{cdr} light: {score}")

    # Step 2: Calculate AP score for each CDR
    for cdr_name, cdr_range in cdr_ranges.items():
        preds_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='heavy'")["prediction"].tolist()
        labs_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='heavy'")["labels"].tolist()
        ap_scores[cdr_name] = average_precision_score(labs_cdr, preds_cdr)
    return ap_scores


# LLM

In [112]:
predictions_llm = pd.read_csv("/home/gathenes/paragraph_benchmark/convex_hull/z_lr-0.00001_dr-0.4,0.3,0.2_mk-0.4_bs-15_dim1-4000,2000,500_alphas-5_pen-0_weight_1_--bigembedding_multi__/prediction.csv")
predictions_llm["IMGT_bis"] = predictions_llm["IMGT"].str.replace(r'[a-zA-Z]$', '', regex=True).astype(int)


In [113]:
preds_cdrs=predictions_llm.query("IMGT_bis in @cdrs")["prediction"].tolist()
labs_cdrs=predictions_llm.query("IMGT_bis in @cdrs")["labels"].tolist()
preds_framework=predictions_llm.query("IMGT_bis not in @cdrs")["prediction"].tolist()
labs_framework=(predictions_llm.query("IMGT_bis not in @cdrs")["labels"]).tolist()
total_preds = preds_cdrs+preds_framework
total_labs=labs_cdrs+labs_framework


In [114]:
import numpy as np
ap = average_precision_score(total_labs, total_preds)
ap_cdrs = average_precision_score(labs_cdrs, preds_cdrs)
ap_framework = average_precision_score(np.array(labs_framework), np.array(preds_framework))
print(ap, ap_cdrs, ap_framework)


0.7270427331778189 0.7589673838367047 0.5186822220129452


In [115]:
# Display the AP scores for each CDR
ap_scores = get_ap_scores(predictions_llm)
print("Average Precision Scores by CDR:")
for cdr, score in ap_scores.items():
    print(f"{cdr} heavy: {score}")


Average Precision Scores by CDR:
cdr1 light: 0.7586930793695318
cdr2 light: 0.5805366580313819
cdr3 light: 0.7587022461526179
Average Precision Scores by CDR:
cdr1 heavy: 0.7270926177009996
cdr2 heavy: 0.7990506958411646
cdr3 heavy: 0.7542836285197275


# Paragraph

In [116]:
predictions_paragraph = pd.read_csv("/home/gathenes/paragraph_benchmark/paragraph_weights/prediction.csv")
predictions_paragraph["IMGT_bis"] = predictions_paragraph["IMGT"].str.replace(r'[a-zA-Z]$', '', regex=True).astype(int)


In [117]:
# Display the AP scores for each CDR
ap_scores = get_ap_scores(predictions_paragraph)
print("Average Precision Scores by CDR:")
for cdr, score in ap_scores.items():
    print(f"{cdr} heavy: {score}")


Average Precision Scores by CDR:
cdr1 light: 0.7604520415881645
cdr2 light: 0.7133327813052373
cdr3 light: 0.7887800763706876
Average Precision Scores by CDR:
cdr1 heavy: 0.7529291932397588
cdr2 heavy: 0.8019699167499572
cdr3 heavy: 0.8173583479659525


# LLM-Paragraph

In [118]:
predictions_llm_paragraph = predictions_llm
df_merged = pd.merge(predictions_llm_paragraph, predictions_paragraph[['pdb', 'IMGT', 'chain_type', 'prediction']],
                    on=["pdb", "IMGT", "chain_type"],
                    how="left",
                    suffixes=("_llm", "_paragraph"))


In [119]:
import numpy as np

# Assuming df_merged is the DataFrame you have after merging
df_merged["prediction"] = np.where(
    df_merged["prediction_paragraph"].notna(),
    df_merged["prediction_paragraph"],
    df_merged["prediction_llm"]
)
df_merged["prediction_paragraph_extended"] = np.where(
    df_merged["prediction_paragraph"].notna(),
    df_merged["prediction_paragraph"],
    0
)


In [120]:
ap_combined = average_precision_score(df_merged["labels"].to_list(), df_merged["prediction"].to_list())
ap_combined_cdrs = average_precision_score(df_merged.query("IMGT_bis in @cdrs")["labels"].to_list(), df_merged.query("IMGT_bis in @cdrs")["prediction"].to_list())
ap_combined_framework = average_precision_score(df_merged.query("IMGT_bis not in @cdrs")["labels"].to_list(), df_merged.query("IMGT_bis not in @cdrs")["prediction"].to_list())


In [121]:
print(ap_combined, ap_combined_cdrs, ap_combined_framework)


0.7544987340781626 0.7895647302238212 0.5396840569636212


In [122]:
ap_paragraph = average_precision_score(df_merged["labels"].to_list(), df_merged["prediction_paragraph_extended"].to_list())
ap_paragraph_cdrs = average_precision_score(df_merged.query("IMGT_bis in @cdrs")["labels"].to_list(), df_merged.query("IMGT_bis in @cdrs")["prediction_paragraph_extended"].to_list())
ap_paragraph_framework = average_precision_score(df_merged.query("IMGT_bis not in @cdrs")["labels"].to_list(), df_merged.query("IMGT_bis not in @cdrs")["prediction_paragraph_extended"].to_list())


In [123]:
print(ap_paragraph, ap_paragraph_cdrs, ap_paragraph_framework)


0.7328245237447634 0.789636884600756 0.4225746917471489


In [124]:
# Display the AP scores for each CDR
ap_scores = get_ap_scores(df_merged)
print("Average Precision Scores by CDR:")
for cdr, score in ap_scores.items():
    print(f"{cdr} heavy: {score}")


Average Precision Scores by CDR:
cdr1 light: 0.7603917973614088
cdr2 light: 0.7133327813052373
cdr3 light: 0.7887800763706876


Average Precision Scores by CDR:
cdr1 heavy: 0.7523288755290967
cdr2 heavy: 0.8019699167499572
cdr3 heavy: 0.8173583479659525


# Paragraph with ground truth crystals

In [125]:
predictions = pd.read_csv("/home/gathenes/paragraph_benchmark/paragraph_weights_2/prediction.csv")
predictions["IMGT_bis"] = predictions["IMGT"].str.replace(r'[a-zA-Z]$', '', regex=True).astype(int)
preds_cdrs=predictions.query("IMGT_bis in @cdrs")["prediction"].tolist()
labs_cdrs=predictions.query("IMGT_bis in @cdrs")["labels"].tolist()
ap_cdrs = average_precision_score(labs_cdrs, preds_cdrs)
print(ap_cdrs)


0.8072495191873574


In [126]:
import pandas as pd
from sklearn.metrics import average_precision_score

# Assuming `predictions` DataFrame is already defined

# Step 1: Convert "IMGT" column to numeric after removing letters
predictions["IMGT_bis"] = predictions["IMGT"].str.replace(r'[a-zA-Z]$', '', regex=True).astype(int)

# Define CDR ranges
cdr_ranges = {
    "cdr1": cdr1,
    "cdr2": cdr2,
    "cdr3": cdr3
}

# Initialize a dictionary to store AP scores for each CDR
ap_scores = {}

for cdr_name, cdr_range in cdr_ranges.items():
    preds_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='light'")["prediction"].tolist()
    labs_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='light'")["labels"].tolist()
    ap_scores[cdr_name] = average_precision_score(labs_cdr, preds_cdr)

# Display the AP scores for each CDR
print("Average Precision Scores by CDR:")
for cdr, score in ap_scores.items():
    print(f"{cdr} light: {score}")

# Step 2: Calculate AP score for each CDR
for cdr_name, cdr_range in cdr_ranges.items():
    preds_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='heavy'")["prediction"].tolist()
    labs_cdr = predictions.query("IMGT_bis in @cdr_range and chain_type=='heavy'")["labels"].tolist()
    ap_scores[cdr_name] = average_precision_score(labs_cdr, preds_cdr)

# Display the AP scores for each CDR
print("Average Precision Scores by CDR:")
for cdr, score in ap_scores.items():
    print(f"{cdr} heavy: {score}")


Average Precision Scores by CDR:
cdr1 light: 0.7804499187675555
cdr2 light: 0.7216139782939159
cdr3 light: 0.7995423740975024
Average Precision Scores by CDR:
cdr1 heavy: 0.7841651978049836
cdr2 heavy: 0.8072048851438248
cdr3 heavy: 0.8423764009933188
